# 12.5 DeepKriging ST

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import xarray as xr
import pandas as pd
import geopandas as gpd
import rasterio as rio

from utils.helpers import *
from utils.metrics import *

import tensorflow as tf
from keras.callbacks import EarlyStopping
import models.stdk as stdk

2025-11-16 19:29:33.534984: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-16 19:29:33.730504: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### 1. Prepare data

In [2]:
root_path = './data/Bangladesh/'

bgd_shp = gpd.read_file(f'{root_path}/BGD_shp/vb025yv7599.shp')
info_file = pd.read_csv(f'{root_path}/target/well_info.csv')

lag=5

In [4]:
cv_setup = np.load(f'{root_path}/cv_setup.npy', allow_pickle=True).item()
train_ids = cv_setup['train_ids']
holdout_ids = cv_setup['holdout_ids']
tr_idxs = cv_setup['tr_idxs']
test_idxs = cv_setup['test_idxs']
seed = cv_setup['seed']

tf.random.set_seed(seed)

In [5]:
full_era_ds = xr.open_mfdataset(f'{root_path}/inputs/PR_AET_M_2002_2017_BGD_025.nc4')
full_era_ds['time'] = [x + pd.DateOffset(days=14) for x in full_era_ds['time'].values ]
full_precip_da = full_era_ds['pr'][(lag-1):]
full_aet_da = full_era_ds['aet'][(lag-1):]

full_twsa_da = xr.open_mfdataset(f'{root_path}/inputs/TWSA_M_2002_2017_BGD_025.nc4')['twsa'][(lag-1):]
full_twsa_da['time'] = [x + pd.DateOffset(days=14) for x in full_twsa_da['time'].values ]

dem_da = xr.open_mfdataset(f'{root_path}/inputs/DEM_BGD_025.nc4')['dem'][0]
full_ndvi_da = xr.open_mfdataset(f'{root_path}/inputs/NDVI_M_2002_2017_BGD_025.nc4')['NDVI'][(lag-1):]
full_ndvi_da['time'] = [x + pd.DateOffset(days=14) for x in full_ndvi_da['time'].values ]

In [6]:
miss_ts = pd.read_csv(f"{root_path}/target/filtered_gws_ts_data_1961_2019.csv", parse_dates=['Date'], index_col='Date')
miss_ts = miss_ts['2002-04-01':'2017-05-31'].dropna(axis=1, how='all')[(lag-1):]
miss_ts_gwsa = miss_ts - miss_ts['2004-01-01':'2009-12-31'].mean()
miss_ts_gwsa

,FA062,RA-63,DI077,BO033,PT005,DI058,RAJ-A-4,FA012,BA003,FA014_1,...,CT011_1,RA-06_1,SY011_1,KH009_1,PA012_1,KT041_1,BO020_1,RA-39_1,MY073,CM016_1
Date,,,,,,,,,,,,,,,,,,,,,
2002-08-15,0.035732,0.266229,0.167378,0.126731,0.004849,0.188272,0.219965,0.181349,0.043500,0.348828,...,0.038336,0.076653,NaN,0.037171,0.221155,0.183615,0.118237,0.248093,0.069225,0.092538
2002-09-15,0.028685,0.192840,0.133967,0.109969,0.004391,0.161430,0.236155,0.169446,0.041755,0.305976,...,0.037981,0.063972,0.014714,0.039259,0.152881,0.235851,0.088037,0.184756,0.092793,NaN
2002-10-15,0.012771,0.187793,0.154595,0.102847,0.004849,0.157403,0.211211,0.122933,0.034546,0.209994,...,0.036050,0.043940,0.013045,0.030290,0.136781,0.187911,0.189162,0.144253,0.097922,0.037301
2002-11-15,-0.001840,0.086861,0.150209,0.067961,0.003961,0.095833,0.136641,0.080829,0.029730,0.066394,...,0.039353,0.006265,0.010918,0.026380,0.041796,0.096614,0.022146,0.049879,0.092824,NaN
2002-12-15,-0.012661,0.015540,NaN,0.045235,0.002334,0.048690,0.073910,0.030656,0.015509,-0.036177,...,0.038387,-0.010128,0.010089,0.019737,-0.018899,0.017024,0.022146,-0.020170,0.082071,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-01-15,-0.001840,NaN,0.009045,-0.055977,NaN,NaN,NaN,-0.030009,-0.001467,-0.079112,...,NaN,-0.104371,-0.045727,0.018628,-0.065246,-0.082508,-0.103230,-0.068590,-0.103694,NaN
2017-02-15,-0.015199,NaN,-0.015468,-0.090689,NaN,NaN,NaN,-0.085384,-0.012483,-0.129631,...,NaN,-0.068350,NaN,-0.007972,NaN,-0.158661,-0.156309,-0.105594,-0.165713,NaN
2017-03-15,-0.014364,NaN,-0.080636,-0.103138,NaN,-0.049119,NaN,-0.104341,-0.017224,-0.165893,...,-0.026186,-0.115581,NaN,-0.016535,-0.158614,-0.183723,-0.165461,-0.208286,-0.188839,NaN


In [7]:
#### prepare training set
coords_train = np.array([ info_file[info_file['SegmentID'] == well][['Latitude', 'Longitude']].values[0] for well in train_ids ])
df_train = miss_ts_gwsa[train_ids]

In [3]:
stdk.build_interpolator_scalar_cov(
    n_space_basis=64, n_time_basis=16,
    ls_space=0.15, ls_time=0.10,
    hidden=(256,128,64), dropout=0.10,
    quantiles=(0.1,0.5,0.9), n_scalar_cov=5
).summary()

Model: "STDK_Interp_ScalarCov"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 coords_xy (InputLayer)      [(None, 2)]                  0         []                            
                                                                                                  
 time_scalar (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 phi_space (RBFEmbed)        (None, 64)                   129       ['coords_xy[0][0]']           
                                                                                                  
 phi_time (RBFEmbed)         (None, 16)                   17        ['time_scalar[0][0]']         
                                                                              

In [11]:
stdk.build_grid_forecaster((12, 24, 24, 5), [0.1, 0.5, 0.9], ~dem_da.isnull().values, hidden=(32, 32)).summary()

Model: "STDK_ConvLSTM_Quantiles_TF_Masked"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 12, 24, 24, 5)]   0         
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 12, 24, 24, 32)    42752     
                                                                 
 conv_lstm2d_3 (ConvLSTM2D)  (None, 24, 24, 32)        73856     
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 3)         99        
                                                                 
Total params: 116707 (455.89 KB)
Trainable params: 116707 (455.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
for k, (tr_idx, val_test_idx) in enumerate(zip(tr_idxs, test_idxs)):
    val_idx, tes_idx = val_test_idx[:int(len(val_test_idx)/2)], val_test_idx[int(len(val_test_idx)/2):]

    fold_df_train = df_train.iloc[np.concatenate([tr_idx, val_idx])]
    cov_das = [full_precip_da[:(val_idx[-1]+1)], full_aet_da[:(val_idx[-1]+1)], full_twsa_da[:(val_idx[-1]+1)], full_ndvi_da[:(val_idx[-1]+1)], dem_da]

    fold_df_train_long = (
        fold_df_train
          .rename_axis(index="date")
          .reset_index()
          .melt(id_vars="date", var_name="station_id", value_name="y")
    )
    
    fold_df_train_long["station_id"] = pd.Categorical(fold_df_train_long["station_id"],
                                        categories=miss_ts.columns,
                                        ordered=True)
    
    fold_df_train_long = fold_df_train_long.sort_values(["date", "station_id"], ignore_index=True)
    
    #### create an interpolation hold-out set
    rng = np.random.default_rng()              # set a seed for reproducibility (optional)
    n_cols = fold_df_train.shape[1]
    idx = rng.choice(n_cols, size=int(0.1*n_cols), replace=False)
    fold_val_locs = idx.tolist()
    fold_val_locs.sort()
    fold_train_locs = [i for i in range(fold_df_train.shape[1]) if i not in fold_val_locs]
    fold_val_ids = fold_df_train.columns[fold_val_locs]
    fold_train_ids = fold_df_train.columns[fold_train_locs]
    
    is_val = fold_df_train_long['station_id'].isin(fold_val_ids.values)
    
    y_train = fold_df_train_long[~is_val]
    y_val   = fold_df_train_long[ is_val]
    
    coords_train = np.array([ info_file[info_file['SegmentID'] == well][['Latitude', 'Longitude']].values[0] for well in fold_train_ids ])
    coords_val = np.array([ info_file[info_file['SegmentID'] == well][['Latitude', 'Longitude']].values[0] for well in fold_val_ids ])
        
    N_train = len(fold_train_ids)
    N_val = len(fold_val_ids)
    T = fold_df_train.shape[0]
    
    ## normalize coordinates to [0,1]^2 over the domain
    lat_min, lat_max = coords_train[:, 0].min(), coords_train[:, 0].max()
    lon_min, lon_max = coords_train[:, 1].min(), coords_train[:, 1].max()
    
    xy01_train = np.vstack([norm_xy(coords_train[i,0], coords_train[i,1], lat_min, lat_max, lon_min, lon_max) for i in range(N_train)])  # [N_train,2]
    xy01_val = np.vstack([norm_xy(coords_val[i,0], coords_val[i,1], lat_min, lat_max, lon_min, lon_max) for i in range(N_val)])  # [N_val,2]
    
    ## normalize time stamps to [0,1] across the whole span
    months = fold_df_train.index.values
    t0 = months.min()
    t1 = months.max()
    t01_all = np.array([norm_time(ts, t0, t1) for ts in months], dtype=np.float32)  # [T]
    
    # build the master design (time-major stacking)
    # coords_xy01_train: [T*N_train, 2]
    coords_xy01_train = np.repeat(xy01_train[None, :, :], T, axis=0).reshape(T*N_train, 2)
    # coords_xy01_val: [T*N_val, 2]
    coords_xy01_val = np.repeat(xy01_val[None, :, :], T, axis=0).reshape(T*N_val, 2)
    # time_scalar: [T*N_train, 1]
    time_scalar_train = np.repeat(t01_all[:, None], N_train, axis=1).reshape(T*N_train, 1).astype(np.float32)
    # time_scalar: [T*N_val, 1]
    time_scalar_val = np.repeat(t01_all[:, None], N_val, axis=1).reshape(T*N_val, 1).astype(np.float32)
    
    ## build y in the EXACT SAME ORDER (time-major)
    # make a dense matrix Ymat[t, i] holding y for station i at month t
    # if y_df is long-form, pivot to [T x N] in the same order
    month_order = list(months)  # already sorted
    
    # Ymat_train shape: [T, N_train] with NaN where missing
    Ymat_train = y_train.pivot(index="date", columns="station_id", values="y").reindex(index=month_order, columns=fold_train_ids).values
    # Ymat_val shape: [T, N_val] with NaN where missing
    Ymat_val = y_val.pivot(index="date", columns="station_id", values="y").reindex(index=month_order, columns=fold_val_ids).values
    
    ## build a DataArray of all station points (vectorized interpolation)
    lon_points_train = xr.DataArray(coords_train[:,1], dims=("station",), coords={"station": fold_train_ids})
    lat_points_train = xr.DataArray(coords_train[:,0], dims=("station",), coords={"station": fold_train_ids})
    lon_points_val = xr.DataArray(coords_val[:,1], dims=("station",), coords={"station": fold_val_ids})
    lat_points_val = xr.DataArray(coords_val[:,0], dims=("station",), coords={"station": fold_val_ids})
    
    y_vec_train = Ymat_train.reshape(-1)                      # [T*N_train]
    y_vec_val = Ymat_val.reshape(-1)                      # [T*N_train]
    
    keep_train = ~np.isnan(y_vec_train)
    keep_val = ~np.isnan(y_vec_val)
    
    ## sample at points
    Cs_keep_train = []
    Cs_keep_val = []
    pred_das_scaled = []
    
    for cov_da in cov_das:
        C_train = cov_da.sel(lon=lon_points_train, lat=lat_points_train, method="nearest")   # [time, station]
        C_val = cov_da.sel(lon=lon_points_val, lat=lat_points_val, method="nearest")   # [time, station]
    
        Cmat_train = C_train.to_numpy() # shape [T, N_train] float32, may contain NaN
        Cmat_val = C_val.to_numpy() # shape [T, N_val] float32, may contain NaN
    
        if cov_da.name == 'dem':
            Cmat_train = np.tile(Cmat_train, T)
            Cmat_val = np.tile(Cmat_val, T)
        
        cov_vec_train = Cmat_train.reshape(-1, 1)                   # [T*N_train, 1]
        cov_vec_val = Cmat_val.reshape(-1, 1)                   # [T*N_train, 1]
    
        C_min, C_max = cov_vec_train[keep_train].min(), cov_vec_train[keep_train].max()
        C_keep_scaled_train = (cov_vec_train[keep_train] - C_min)/(C_max - C_min)
        C_keep_scaled_val = (cov_vec_val[keep_val] - C_min)/(C_max - C_min)
    
        Cs_keep_train.append(C_keep_scaled_train)
        Cs_keep_val.append(C_keep_scaled_val)
        # prepare pred covariates
        pred_das_scaled.append((cov_da - C_min) / (C_max - C_min))
    
    covs_train = np.concatenate(Cs_keep_train, axis=1)
    covs_val = np.concatenate(Cs_keep_val, axis=1)
    
    inputs_train = {
        "coords_xy":  coords_xy01_train[keep_train].astype("float32"),
        "time_scalar": time_scalar_train[keep_train].astype("float32"),
        "cov_scalar_vec": covs_train.astype("float32"),   # ← scalar cov input
    }
    
    inputs_val = {
        "coords_xy":  coords_xy01_val[keep_val].astype("float32"),
        "time_scalar": time_scalar_val[keep_val].astype("float32"),
        "cov_scalar_vec": covs_val.astype("float32"),   # ← scalar cov input
    }
    
    target_train = y_vec_train[keep_train].astype("float32")
    target_val = y_vec_val[keep_val].astype("float32")

    model_ipol = stdk.build_interpolator_scalar_cov(
        n_space_basis=64, n_time_basis=16,
        ls_space=0.15, ls_time=0.10,
        hidden=(256,128,64), dropout=0.10,
        quantiles=(0.1,0.5,0.9), n_scalar_cov=5
    )
    
    history = model_ipol.fit(inputs_train, target_train,
                          batch_size=2048, epochs=50,
                          validation_data=[inputs_val, target_val], verbose=1)
    
    T, H, W = len(months), full_precip_da.sizes["lat"], full_precip_da.sizes["lon"]
    lon_grid = full_precip_da["lon"].to_numpy()
    lat_grid = full_precip_da["lat"].to_numpy()
    
    Lon, Lat = np.meshgrid(lon_grid, lat_grid)              # [H,W]
    X01, Y01 = norm_xy(Lat, Lon, lat_min, lat_max, lon_min, lon_max)
    coords_xy01_grid = np.stack([X01.ravel(), Y01.ravel()], axis=1).astype("float32")  # [H*W, 2]

    pred_da = stdk.predict_all_months_scalar_cov(model_ipol, pred_das_scaled, months, coords_xy01_grid, t0, t1)

    land_mask_da = ~cov_das[0][0].isnull()
    
    in_len  = 12   # use past 12 frames as context
    out_len = 1   # 1-step ahead target for training
    n = cov_das[0].shape[0]-in_len
    
    y_q50 = pred_da.sel(quantile="0.5")
    
    da_fit = y_q50[tr_idx]  # if you really want to fit on ALL available time
    
    da_std = y_q50.transpose('time','lat','lon')
    land_vals = da_fit.where(land_mask_da).values
    vmin = np.nanmin(land_vals)
    vmax = np.nanmax(land_vals)
    rng  = max(vmax - vmin, 1e-8)
    print("Anomaly training vmin/vmax:", vmin, vmax)
    
    norm = ((da_std - vmin) / rng).clip(0, 1)
    norm = norm.where(land_mask_da, -1.0)   # ocean sentinel
    
    X_targ = norm.values.astype('float32')           # (T, H, W)
    lm_np  = land_mask_da.values                                # (H, W)
    
    # Optional: stack exogenous channels (must match grid/time)
    # da_exog: (time, channel, y, x)
    cov_stack = stack_covariates(pred_das_scaled, months)
    ex = cov_stack.values.astype("float32")  # (T,H,W,C)
    ex_ = np.empty_like(ex)
    
    mins, maxs = [], []
    for c in range(ex.shape[-1]):
        e   = ex[..., c]                                  # (T,H,W)
        ev  = np.where(lm_np, e, np.nan)[tr_idx]
        cmin = float(np.nanmin(ev)); cmax = float(np.nanmax(ev)); crng = max(cmax - cmin, 1e-8)
        mins.append(cmin); maxs.append(cmax)
        e_norm = ((e - cmin) / crng).clip(0.0, 1.0)
        e_norm = np.where(lm_np[None, ...], e_norm, -1.0) # ocean = -1
        ex_[..., c] = e_norm
    
    print("Exog fitted mins:", mins)
    print("Exog fitted maxs:", maxs)
    
    # Final model input (time, lat, lon, channels)
    X_all = np.concatenate([X_targ[..., None], ex_], axis=-1)  # (T,H,W,C_total)
    
    T,H,W,C = X_all.shape
    
    # target channel
    land_norm = X_all[..., 0][:, lm_np]  # (T, Nland)
    print("Target land min/max (expect ~0..1):", float(land_norm.min()), float(land_norm.max()))
    ocean_norm = X_all[..., 0][:, ~lm_np]
    print("Target ocean unique (expect [-1]):", np.unique(ocean_norm))
    
    # exogenous channels
    ex_land_min = min(float(X_all[..., i][:, lm_np].min()) for i in range(1,C))
    ex_land_max = max(float(X_all[..., i][:, lm_np].max()) for i in range(1,C))
    print("Exog land min/max (expect ~0..1):", ex_land_min, ex_land_max)
    
    # ---- (B) sliding windows ----
    
    def make_windows(X, in_len, out_len):
        T = X.shape[0]
        X_in, Y_out = [], []
        for t in range(T - in_len - out_len + 1):
            X_in.append(X[t:t+in_len])                        # (in_len, H, W, C)
            Y_out.append(X[t+in_len:t+in_len+out_len, ..., 0:1])  # target channel only -> (out_len,H,W,1)
        X_in  = np.stack(X_in)                                # (N, in_len, H, W, C)
        Y_out = np.stack(Y_out)                               # (N, out_len, H, W, 1)
        return X_in, Y_out
    
    X_win, Y_win = make_windows(X_all, in_len, out_len)
    
    # For 1-step training with quantile loss, use the last target frame only: (N,H,W,1)
    Y_last = Y_win[:, -1, ...]   # (N, H, W, 1)
    
    # train/val split on the **time** axis
    X_tr, Y_tr = X_win[:-len(val_idx)], Y_last[:-len(val_idx)]
    X_va, Y_va = X_win[-len(val_idx):], Y_last[-len(val_idx):]
    
    # tf.data pipelines
    batch_size = 8
    train_ds = tf.data.Dataset.from_tensor_slices((X_tr, Y_tr)).shuffle(2048).batch(batch_size).prefetch(2)
    val_ds   = tf.data.Dataset.from_tensor_slices((X_va, Y_va)).batch(batch_size).prefetch(2)
    
    # Quantiles to predict
    quantiles = [0.1, 0.5, 0.9]

    scaler = {
        "target": {"vmin": vmin, "vmax": vmax},
        "exog":   [{"vmin": mn, "vmax": mx} for mn, mx in zip(mins, maxs)],
    }
    model = stdk.build_grid_forecaster((in_len, H, W, C), quantiles, lm_np, hidden=(32, 32))
    model.summary()

    callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss')
    ]
    history = model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=callbacks)

    steps = len(tes_idx)
    pred_seq_norm, forecast_times = stdk.multi_step_forecast(model, X_all, steps, in_len, lm_np, quantiles, time_ref=pred_da.time.values)
    # pred_seq shape should be (steps, H, W, Q)
    assert pred_seq_norm.ndim == 4
    steps_, H_, W_, Q_ = pred_seq_norm.shape
    assert Q_ == len(quantiles)
    
    pred_seq_norm = np.where(land_mask_da.values[None, ..., None], pred_seq_norm, np.nan)
    
    rng = scaler["target"]["vmax"] - scaler["target"]["vmin"]
    pred_seq_phys = pred_seq_norm * rng + scaler["target"]["vmin"]
    
    pred_seq_da = xr.DataArray(
        pred_seq_phys, dims=("time", "lat", "lon", "quantile"),
        coords={"time": forecast_times, "lat": pred_da.lat, "lon": pred_da.lon, "quantile": quantiles},
        name="forecast"
    )
    pred_da = pred_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat', inplace=True)
    pred_da = pred_da.rio.write_crs("epsg:4326", inplace=True)
    pred_da = pred_da.rio.clip(bgd_shp.geometry.values, bgd_shp.crs, all_touched=True)
    
    pred_seq_da = pred_seq_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat', inplace=True)
    pred_seq_da = pred_seq_da.rio.write_crs("epsg:4326", inplace=True)
    pred_seq_da = pred_seq_da.rio.clip(bgd_shp.geometry.values, bgd_shp.crs, all_touched=True)

    pred_da.to_dataset(name='gwsa_pred').to_netcdf(f'{root_path}/outputs/DeepKriging/st_deepkrig_all_cov_ipol_fold_{k}.nc4')
    pred_seq_da.to_dataset(name='gwsa_pred').to_netcdf(f'{root_path}/outputs/DeepKriging/st_deepkrig_all_cov_fcst_fold_{k}.nc4')

Epoch 1/50
8/8 [==============================] - 1s 28ms/step - loss: 0.0440 - val_loss: 0.0274
Epoch 2/50
8/8 [==============================] - 0s 10ms/step - loss: 0.0274 - val_loss: 0.0205
Epoch 3/50
8/8 [==============================] - 0s 11ms/step - loss: 0.0225 - val_loss: 0.0172
Epoch 4/50
8/8 [==============================] - 0s 11ms/step - loss: 0.0203 - val_loss: 0.0157
Epoch 5/50
8/8 [==============================] - 0s 10ms/step - loss: 0.0189 - val_loss: 0.0149
Epoch 6/50
8/8 [==============================] - 0s 10ms/step - loss: 0.0181 - val_loss: 0.0143
Epoch 7/50
8/8 [==============================] - 0s 10ms/step - loss: 0.0175 - val_loss: 0.0140
Epoch 8/50
8/8 [==============================] - 0s 10ms/step - loss: 0.0172 - val_loss: 0.0138
Epoch 9/50
8/8 [==============================] - 0s 11ms/step - loss: 0.0170 - val_loss: 0.0137
Epoch 10/50
8/8 [==============================] - 0s 10ms/step - loss: 0.0165 - val_loss: 0.0134
Epoch 11/50
8/8 [============